## Define connection to IBM Cloud Object Storage

In [ ]:
cos_api_key=''
cos_endpoint_url=''
cos_bucket = ''
cos_object_key = ''

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id=cos_api_key,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url=cos_endpoint_url
)

body = cos_client.get_object(Bucket=cos_bucket,Key=cos_object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body, sep=';', parse_dates=['Order_Date'], dayfirst=True)
df_1.head(10)


In [ ]:
df_1.dtypes

## Create prompt to convert user query to SQL statement

In [ ]:
table_name = 'online_transactions'

In [ ]:
system_prompt = """
You are assistant to convert user query to SQL statement.
SQL statement must use sqllite3 from python module.
SQL statement must refer to table schema as below:

Database Schema:
Table Name: online_transactions
Columns:
 - name: Order_Number, data_type: text
 - name: State_Code, data_type: text
 - name: Customer_Name, data_type: text
 - name: Order_Date, data_type: date
 - name: Status, data_type: text
 - name: Product, data_type: text
 - name: Category, data_type: text
 - name: Brand, data_type: text
 - name: Cost, data_type: numeric
 - name: Sales, data_type: numeric
 - name: Quantity, data_type: numeric
 - name: Total_Cost, data_type: numeric
 - name: Total_Sales, data_type: numeric
 - name: Supervisor, data_type: text

You must generate the answer in JSON format using the structure below:

JSON Format:
```
{"result": "<SQL statement>"}
```

Sample query 1: display top 5 states ordered by total sales
Output: {"result": "SELECT State_Code, SUM(Total_Sales) as Total_Sales FROM online_transactions GROUP BY State_Code ORDER BY SUM(Total_Sales) DESC LIMIT 5"}

Sample query 2: perform trend analysis of cost and sales from 1 January 2022 to 31 December 2022 on monthly basis
Output: {"result": "SELECT strftime('%m', Order_Date) as Order_Date_Month, SUM(Total_Sales) as Total_Sales, SUM(Total_Cost) as Total_Cost FROM online_transactions WHERE Order_Date between '2022-01-01' and '2022-12-31' GROUP BY strftime('%m', Order_Date) ORDER BY strftime('%m', Order_Date) ASC"}

Be straightforward and remember to geenrate answer in JSON format!
"""

In [ ]:
watsonx_api_key=""
watsonx_project_id=""
watsonx_url=""
watsonx_space_id=""
model_id=""

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

import json

In [ ]:
credentials = Credentials(
    url=watsonx_url,
    api_key=watsonx_api_key,
)

In [ ]:
llm_model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=watsonx_project_id,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 4000,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.TEMPERATURE: 0.9,
        GenParams.RANDOM_SEED: 334,
    }
)

In [ ]:
input_prompt = "siapa customer yang paling banyak membeli?"

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": input_prompt}
]

In [ ]:
llm_result = llm_model.chat(messages=messages)
sql_json = json.loads(llm_result["choices"][0]["message"]["content"])
sql_statement = sql_json["result"]
sql_statement

## Perform query to dataframe

In [ ]:
import sqlite3

df = df_1.copy()

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Store the DataFrame as a table in the database
df.to_sql('online_transactions', conn, index=False, if_exists='replace')

# Write and execute the SQL query
query = sql_statement
result = pd.read_sql_query(query, conn)
result = result.to_dict(orient='records')

# Display the result
result

## Compile functions into single function

In [ ]:
input_prompt = "siapa customer yang paling banyak membeli?"

payload = {
    "input_data": [{
        "fields": ["query"],
        "values": [[input_prompt]]
}]}

In [ ]:
def handle_user_query():
    import os, types
    import pandas as pd
    from botocore.client import Config
    import ibm_boto3

    from ibm_watsonx_ai.foundation_models import ModelInference
    from ibm_watsonx_ai import Credentials
    from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
    
    import json
    import sqlite3

    cos_api_key=''
    cos_endpoint_url=''
    cos_bucket = ''
    cos_object_key = ''

    watsonx_api_key=""
    watsonx_project_id=""
    watsonx_url=""
    model_id=""

    credentials = Credentials(
        url=watsonx_url,
        api_key=watsonx_api_key,
    )

    llm_model = ModelInference(
        model_id=model_id,
        credentials=credentials,
        project_id=watsonx_project_id,
        params={
            GenParams.DECODING_METHOD: "greedy",
            GenParams.MAX_NEW_TOKENS: 4000,
            GenParams.MIN_NEW_TOKENS: 1,
            GenParams.TEMPERATURE: 0.9,
            GenParams.RANDOM_SEED: 334,
        }
    )

    def retrieve_data_from_cos():
        cos_client = ibm_boto3.client(
            service_name='s3',
            ibm_api_key_id=cos_api_key,
            ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
            config=Config(signature_version='oauth'),
            endpoint_url=cos_endpoint_url
        )
        
        body = cos_client.get_object(Bucket=cos_bucket,Key=cos_object_key)['Body']
        # add missing __iter__ method, so pandas accepts body as file-like object
        if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
        
        df_1 = pd.read_csv(body, sep=';', parse_dates=['Order_Date'], dayfirst=True)
        
        return df_1

    def convert_user_query_to_sql(input_prompt):
        system_prompt = """
        You are assistant to convert user query to SQL statement.
        SQL statement must use sqllite3 from python module.
        SQL statement must refer to table schema as below:
        
        Database Schema:
        Table Name: online_transactions
        Columns:
         - name: Order_Number, data_type: text
         - name: State_Code, data_type: text
         - name: Customer_Name, data_type: text
         - name: Order_Date, data_type: date
         - name: Status, data_type: text
         - name: Product, data_type: text
         - name: Category, data_type: text
         - name: Brand, data_type: text
         - name: Cost, data_type: numeric
         - name: Sales, data_type: numeric
         - name: Quantity, data_type: numeric
         - name: Total_Cost, data_type: numeric
         - name: Total_Sales, data_type: numeric
         - name: Supervisor, data_type: text
        
        You must generate the answer in JSON format using the structure below:
        
        JSON Format:
        ```
        {"result": "<SQL statement>"}
        ```
        
        Sample query 1: display top 5 states ordered by total sales
        Output: {"result": "SELECT State_Code, SUM(Total_Sales) as Total_Sales FROM online_transactions GROUP BY State_Code ORDER BY SUM(Total_Sales) DESC LIMIT 5"}
        
        Sample query 2: perform trend analysis of cost and sales from 1 January 2022 to 31 December 2022 on monthly basis
        Output: {"result": "SELECT strftime('%m', Order_Date) as Order_Date_Month, SUM(Total_Sales) as Total_Sales, SUM(Total_Cost) as Total_Cost FROM online_transactions WHERE Order_Date between '2022-01-01' and '2022-12-31' GROUP BY strftime('%m', Order_Date) ORDER BY strftime('%m', Order_Date) ASC"}
        
        Be straightforward and remember to geenrate answer in JSON format!
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_prompt}
        ]

        llm_result = llm_model.chat(messages=messages)
        sql_json = json.loads(llm_result["choices"][0]["message"]["content"])
        sql_statement = sql_json["result"]

        return sql_statement

    def retrieve_data_from_dataframe(df, sql_statement):
        conn = sqlite3.connect(':memory:')
        
        df.to_sql('online_transactions', conn, index=False, if_exists='replace')
        
        result = pd.read_sql_query(sql_statement, conn)
        result = result.to_dict(orient='records')
        
        return result

    def handle_user_query(payload):
        input_prompt = payload["input_data"][0]["values"][0][0] # get user query
        sql_statement = convert_user_query_to_sql(input_prompt) # get SQL statement
        df = retrieve_data_from_cos() # get pandas dataframe
        result = retrieve_data_from_dataframe(df, sql_statement)
        
        return {
            'predictions': 
            [
                {
                    'fields': ['result'], 
                    'values': [[result]]
                }
            ]
        }

    return handle_user_query

In [ ]:
response = handle_user_query()(payload)
response

## Deploy function

In [ ]:
watsonx_api_key=""
watsonx_project_id=""
watsonx_url=""
watsonx_space_id=""
model_id=""

creds = {
    "url": watsonx_url,
    "apikey": watsonx_api_key 
}

In [ ]:
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(creds)

### Import asset to deployment space

In [ ]:
client.set.default_space(watsonx_space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_data = {
    client.repository.FunctionMetaNames.NAME:'handle_user_query',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=handle_user_query)

### Create online deployment

In [ ]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: 'handle_user_query',
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)

In [ ]:
# Get the endpoint URL of the function deployment just created
# function_deployment_id = '8e10d3ab-1b6e-4449-a62b-90966ef0f559'
# function_deployment_endpoint_url = f"https://jp-tok.ml.cloud.ibm.com/ml/v4/deployments/{function_deployment_id}/predictions"
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
client.set.default_space(watsonx_space_id)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))